In [1]:
import numpy as np
import pandas as pd

# Descobrindo as fórmulas da física

Um estagiário alienígena de outro universo recebeu uma bolsa de iniciação científica para descobrir algumas leis da física de nosso universo.

Ele possui apenas uma hora em nosso universo devido a contingência de gastos de seu planeta de origem, e decide então apenas coletar dados de sensores.

## Tarefa 1

Voltando para sua casa, nosso amigo 👽 deseja descobrir a lei que descreve a força resultante de objetos no espaço.

Para este experimento, ele usará os seguintes dados de seus sensores:

- $f$ : Força resultante (em Newton)
- $m$ : Massa de um objeto observado (em $kg$)
- $a$: Aceleração do objeto (em $m$/$s^2$)

In [2]:
a = np.array([1000, 45, 350, 4, 0.1, 58.6, 0.12])
m = np.array([3, 38, 0.1, 500, 4777, 3.56, 0.25])
f = np.array([3000.440045, 1710.059296, 34.67584531, 1999.505549, 477.948657, 208.3847164, -0.02876546])

### Modelo linear

👽 primeiro supõe um modelo linear para descrever o problema

Para o ajudar, implemente uma função que retorna a pseudo-inversa de uma matriz:
$$(A^T A)^{-1} A^T$$

In [3]:
# Pseudo-inversa de A
def pinv(A):
    linhas, colunas = A.shape
    AT = np.zeros((colunas,linhas))
    for i in range(linhas):
        for j in range(colunas):
            AT[j,i] = A[i,j]
            
    try:
        AT_A_inv = np.linalg.inv(AT @ A)
    except Exception as e:
        print("Erro: ",e)
    
    pseudo_A = AT_A_inv @ AT
    
    return pseudo_A

Agora o ajude a resolver o modelo linear

In [4]:
A = np.column_stack((np.ones_like(f), a, m))

# modelo f = A * x
x = pinv(A) @ f

print(x)

[ 6.79074930e+02  1.87938961e+00 -1.17158937e-02]


In [5]:
# Erro relativo das primeiras 5 observações
res = pd.DataFrame({
    'f': f,
    'f_est': A @ x,
    'erro': f - A @ x,
    'erro_relativo': (f - A @ x) / (A @ x)})
res

,f,f_est,erro,erro_relativo
0,3000.440045,2558.429394,442.010651,0.172766
1,1710.059296,763.202259,946.857037,1.240637
2,34.675845,1336.860123,-1302.184277,-0.974062
3,1999.505549,680.734542,1318.771007,1.937276
4,477.948657,623.296045,-145.347388,-0.233192
5,208.384716,789.165453,-580.780737,-0.735943
6,-0.028765,679.297528,-679.326294,-1.000042


Note que o valor estimado é muito diferente do real

In [6]:
# Erro RMSE
(res.erro**2).mean()**0.5   # np.mean((f - A @ x)**2)**0.5

873.9621490734941

Observe que, como um típico aluno de iniciação científica, ele não testou nenhuma das suposições para modelos lineares e nem verificou a significância estatística das variáveis.

### Modelo quadrático

Não gostando do resultado, ele tenta um modelo não linear:

$f(m, a) = c_0 + c_1 a + c_2 m + c_3 a^2 + c_4 m^2 + c_5 m a$

In [7]:
A = pd.DataFrame({
    'c': np.ones_like(f),
    'a': a,
    'm': m,
    'a2': a*a,
    'm2': m*m,
    'ma': m*a})
A

,c,a,m,a2,m2,ma
0,1.0,1000.00,3.00,1000000.0000,9.000000e+00,3000.000
1,1.0,45.00,38.00,2025.0000,1.444000e+03,1710.000
2,1.0,350.00,0.10,122500.0000,1.000000e-02,35.000
3,1.0,4.00,500.00,16.0000,2.500000e+05,2000.000
4,1.0,0.10,4777.00,0.0100,2.281973e+07,477.700
5,1.0,58.60,3.56,3433.9600,1.267360e+01,208.616
6,1.0,0.12,0.25,0.0144,6.250000e-02,0.030


In [10]:
A = np.column_stack((np.ones_like(f), a, m,a*a,m*m,m*a))

# modelo f = A * x

x = pinv(A) @ f

# Erro relativo das primeiras 5 observações
res = pd.DataFrame({
    'f': f,
    'f_est': A @ x,
    'erro': f - A @ x,
    'erro_relativo': (f - A @ x) / (A @ x)})
res

,f,f_est,erro,erro_relativo
0,3000.440045,3000.441979,-0.001934,-6.444443e-07
1,1710.059296,1710.045294,0.014002,8.188101e-06
2,34.675845,34.657911,0.017935,5.174754e-04
3,1999.505549,1999.506075,-0.000526,-2.631029e-07
4,477.948657,477.948652,0.000005,1.030469e-08
5,208.384716,208.469668,-0.084951,-4.074992e-04
6,-0.028765,-0.084235,0.055469,-6.585086e-01


Perceba agora os erros muito pequenos. Vamos ver os coeficientes

In [11]:
x.round(2)

array([-0.11, -0.  , -0.  ,  0.  ,  0.  ,  1.  ])

Ou seja, resulta na fórmula: $f = -0.11 + m * a$

Muito próximo da fórmula real $f = m * a$

A diferença é devido aos sensores de baixa qualidade do Alien e poucas medições.

In [12]:
# Erro RMSE
(res.erro**2).mean()**0.5   # np.mean((f - A @ x)**2)**0.5

0.039306996941771144

## Tarefa 2

Depois ele tenta descobrir a energia potencial gravitacional $e$ em Joules, através dos sensores

- $m$ : Massa de um objeto observado (em $kg$)
- $h$: Altura (em $m$)

Desta, vez, ela já tenta direto um modelo quadrático

In [13]:
m = np.array([3, 38, 0.1, 500, 4777, 3.56, 0.25])
h = np.array([345, 1, 56789, 5561, 67, 0.5, 920])
e = np.array([10142.49025, 372.3987424, 55653.20951, 27248900.1, 3136578.45, 17.16789564, 2253.931423])

In [14]:
# complete abaixo
A = pd.DataFrame({
    'c': np.ones_like(e),
    'm': m,
    'h': h,
    'm2': m*m,
    'h2': h*h,
    'mh': m*h,})
A

,c,m,h,m2,h2,mh
0,1.0,3.00,345.0,9.000000e+00,1.190250e+05,1035.00
1,1.0,38.00,1.0,1.444000e+03,1.000000e+00,38.00
2,1.0,0.10,56789.0,1.000000e-02,3.224991e+09,5678.90
3,1.0,500.00,5561.0,2.500000e+05,3.092472e+07,2780500.00
4,1.0,4777.00,67.0,2.281973e+07,4.489000e+03,320059.00
5,1.0,3.56,0.5,1.267360e+01,2.500000e-01,1.78
6,1.0,0.25,920.0,6.250000e-02,8.464000e+05,230.00


In [16]:
A = np.column_stack((np.ones_like(e), m, h,m*m,h*h,m*h))

# modelo f = A * x
x = pinv(A) @ e

# Erro relativo das primeiras 5 observações
res = pd.DataFrame({
    'e': e,
    'f_est': A @ x,
    'erro': e - A @ x,
    'erro_relativo': (e - A @ x) / (A @ x)})
res

,e,f_est,erro,erro_relativo
0,1.014249e+04,1.014270e+04,-0.209261,-2.063172e-05
1,3.723987e+02,3.723955e+02,0.003280,8.808840e-06
2,5.565321e+04,5.565321e+04,-0.000010,-1.847906e-10
3,2.724890e+07,2.724890e+07,0.000075,2.759971e-12
4,3.136578e+06,3.136578e+06,0.000041,1.293620e-11
5,1.716790e+01,1.704077e+01,0.127122,7.459847e-03
6,2.253931e+03,2.253853e+03,0.078816,3.496926e-05


Resultados muito próximos, estimado e reais. Vamos ver os coeficientes

In [17]:
x.round(2)

array([-0.44,  0.01,  0.  , -0.  , -0.  ,  9.8 ])

Resulta em $e = -0.44 + 9.8 m h$

Na verdade o aluno encontrou $e = -0.44 + 0.01 m+ 9.8 m h$   Que ainda é muito parecido

Muito próximo da lei: $e = g m h$, com aceleração da gravidade $g$ = 9.8 $m/s^2$.

Observando a tabela da aceleração da gravidade em vários planetas em https://pt.wikipedia.org/wiki/Gravidade, tente descobrir de tal planeta ele capturou os dados.

In [18]:
# Erro RMSE
(res.erro**2).mean()**0.5   # np.mean((f - A @ x)**2)**0.5

0.09722790430986539